In [1]:
filePath = "/FileStore/tables/Group/" #put your own file path if necessary

#Importing the files one by one
Complaints = spark.read\
  .format("csv")\
  .option("inferSchema","true")\
  .option("header","true")\
  .option("delimiter",",")\
  .option("0","NA")\
  .load(filePath + "BDT2_1920_Complaints.csv")\

Delivery=spark.read\
  .format("csv")\
  .option("header","true")\
  .option("inferSchema","true")\
  .option("delimiter",",")\
  .load(filePath + "BDT2_1920_Delivery.csv")

Subscriptions=spark.read\
  .format("csv")\
  .option("header","true")\
  .option("inferSchema","true")\
  .option("delimiter",",")\
  .load(filePath + "BDT2_1920_Subscriptions.csv")

Customers=spark.read\
  .format("csv")\
  .option("header","true")\
  .option("inferSchema","true")\
  .option("delimiter",",")\
  .load(filePath + "BDT2_1920_Customers.csv")

Formula=spark.read\
  .format("csv")\
  .option("header","true")\
  .option("inferSchema","true")\
  .option("delimiter",",")\
  .load(filePath + "BDT2_1920_Formula.csv")

In [2]:
from pyspark.sql.functions import *

#Replacing NA in Complaints
#replacing NA with meaningfull value when possible.
#unknown ID will take the value 0
#unknown numeric values like quantities will take the value 999
#NA values in string type column will take a "NA" value or a "no response"/"no solution" depending on the context

#Replacing NA in Complaints
Complaints = Complaints.withColumn("ProductID", when(Complaints["ProductID"] == "NA", 0).otherwise(Complaints["ProductID"]))\
  .withColumn("ProductName", when(Complaints["ProductName"] == "NA", "NA").otherwise(Complaints["ProductName"]))\
  .withColumn("FeedbackTypeID", when(Complaints["FeedbackTypeID"] == "NA", 0).otherwise(Complaints["FeedbackTypeID"]))\
  .withColumn("FeedbackTypeDesc", when(Complaints["FeedbackTypeDesc"] == "NA", "no response").otherwise(Complaints["FeedbackTypeDesc"]))\
  .withColumn("SolutionTypeID", when(Complaints["SolutionTypeID"] == "NA", 0).otherwise(Complaints["SolutionTypeID"]))\
  .withColumn("SolutionTypeDesc", when(Complaints["SolutionTypeDesc"] == "NA", "no solution").otherwise(Complaints["SolutionTypeDesc"]))

#Replacing NA in Delivery
Delivery = Delivery.na.fill("NA", "DeliveryClass")

#Replacing NA in Subscriptions
#NbrMeals_EXCEP NAs have been replaced by the mean NbrMeals_EXCEP ordered by the same NbrMeals_REG category
Subscriptions = Subscriptions.withColumn("NbrMeals_EXCEP",when((Subscriptions["NbrMeals_EXCEP"] == "NA") & (Subscriptions["NbrMeals_REG"]==76), 12).otherwise(Subscriptions["NbrMeals_EXCEP"]))
Subscriptions = Subscriptions.withColumn("NbrMeals_EXCEP",when((Subscriptions["NbrMeals_EXCEP"] == "NA") & (Subscriptions["NbrMeals_REG"]==304), 13).otherwise(Subscriptions["NbrMeals_EXCEP"]))
Subscriptions = Subscriptions.withColumn("NbrMeals_EXCEP",when((Subscriptions["NbrMeals_EXCEP"] == "NA") & (Subscriptions["NbrMeals_REG"]==329), 17).otherwise(Subscriptions["NbrMeals_EXCEP"]))
Subscriptions = Subscriptions.withColumn("NbrMeals_EXCEP",when((Subscriptions["NbrMeals_EXCEP"] == "NA") & (Subscriptions["NbrMeals_REG"]==152), 13).otherwise(Subscriptions["NbrMeals_EXCEP"]))

#RenewalDate 1 and 0 (so if a client renewed 6 times his subscription, the value can be summed to 6)
Subscriptions = Subscriptions.withColumn("RenewalDate",when(Subscriptions["RenewalDate"] == "NA",0).otherwise(1))

#PaymentDate Redondant with PaymentStatus
  #GrossFormulaPrice
  #NetFormulaPrice
  #NbrMealsPrice
  #ProductDiscount
  #FormulaDiscount
  #TotalDiscount
  #TotalPrice
  #TotalCredit
  #All of those are codependent. Maybe after grouping the NbrMeals_REG/EXCEP features, we can replace NA's by the mean of 
  # the category they belong to


In [3]:
#Complaints
Complaints = Complaints.withColumn("ProductID", Complaints["ProductID"].cast("integer"))\
  .withColumn("SolutionTypeID", Complaints["SolutionTypeID"].cast("integer"))\
  .withColumn("FeedbackTypeID", Complaints["FeedbackTypeID"].cast("integer"))

#Subscriptions
Subscriptions = Subscriptions.withColumn("NbrMeals_EXCEP",Subscriptions["NbrMeals_EXCEP"].cast("integer"))
Subscriptions = Subscriptions.withColumn("RenewalDate",Subscriptions["RenewalDate"].cast("integer"))
  #converting timestamps to number of days
Subscriptions = Subscriptions.withColumn("EndDate",Subscriptions["EndDate"].cast("long")/86400)
Subscriptions = Subscriptions.withColumn("StartDate",Subscriptions["StartDate"].cast("long")/86400)
Complaints = Complaints.withColumn("ComplaintDate",Complaints["ComplaintDate"].cast("long")/86400)

In [4]:
#Subscriptions
Subscriptions = Subscriptions.withColumn("SubscriptionDuration", Subscriptions.EndDate - Subscriptions.StartDate)
Subscriptions = Subscriptions.withColumn("NbrMealsPerDay", when(Subscriptions["SubscriptionDuration"] == 0, Subscriptions.NbrMeals_REG).otherwise(Subscriptions.NbrMeals_REG / Subscriptions.SubscriptionDuration))

In [5]:
display(Subscriptions)

SubscriptionID,CustomerID,StartDate,EndDate,NbrMeals_REG,NbrMeals_EXCEP,RenewalDate,PaymentType,PaymentStatus,PaymentDate,FormulaID,GrossFormulaPrice,NetFormulaPrice,NbrMealsPrice,ProductDiscount,FormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,ProductName,SubscriptionDuration,NbrMealsPerDay
627529,775138,17135.0,17225.0,76,10,1,BT,Paid,2016-12-01,919,1480,1480,19.47368,0,0,0,1480,0,Custom Events,90.0,0.8444444444444444
637001,194809,17039.0,17227.0,152,25,1,BT,Paid,2016-08-22,4192,2760,1760,11.57894,0,1000,1000,1760,0,Custom Events,188.0,0.8085106382978723
1238870,654824,17860.0,17950.0,76,10,1,BT,Paid,2018-11-09,10961,1580,1580,20.78948,0,0,0,1580,0,Custom Events,90.0,0.8444444444444444
315743,626815,16801.0,17165.0,304,10,1,BT,Paid,2015-12-18,896,4980,4980,16.38158,0,0,0,4980,0,Custom Events,364.0,0.8351648351648352
1176762,1016426,17760.0,17788.0,25,25,0,BT,Paid,2018-08-13,12867,540,300,12,0,240,240,300,0,Custom Events,28.0,0.8928571428571429
916472,871676,17491.0,17581.0,76,10,1,BT,Paid,2017-11-20,5100,1540,1540,20.26316,0,0,0,1540,0,Custom Events,90.0,0.8444444444444444
646275,655981,17099.0,17130.0,25,10,1,DD,Paid,2016-10-15,924,520,520,20.8,0,0,0,520,0,Custom Events,31.0,0.8064516129032258
752611,704300,17230.0,17258.0,25,25,1,BT,Paid,2017-03-24,5389,458,458,18.32,0,0,0,458,0,Custom Events,28.0,0.8928571428571429
1079202,684448,17652.0,17684.0,25,25,1,DD,Paid,2018-04-21,9466,472,472,18.88,0,0,0,472,0,Custom Events,32.0,0.78125
669473,276941,17122.0,17487.0,304,25,1,BT,Paid,2016-11-24,891,4980,4980,16.38158,0,0,0,4980,0,Custom Events,365.0,0.8328767123287671


In [6]:
Subscriptions.createOrReplaceTempView("subscriptions")

In [7]:
SubInter = spark.sql("select CustomerID, sum(NbrMeals_REG) as TotalMeal_REG, avg(NbrMeals_REG) as MeanMeal_REGPerSub, sum(NbrMeals_EXCEP) as TotalMeal_EXCEP, avg(NbrMeals_EXCEP) as MeanMeal_EXCEPPerSub, min(StartDate) as FirstSubDate, max(EndDate) as EndOfLastSub, (max(EndDate)-min(StartDate)) as HasBeenClientForXDays,count(SubscriptionID) as NbrSub, SUM(CASE WHEN PaymentStatus='Paid' THEN 1 ELSE 0 END) as SubPaid, SUM(CASE WHEN PaymentStatus='Not Paid' THEN 1 ELSE 0 END) as SubNotPaid, SUM(CASE WHEN PaymentStatus='Paid' THEN 1 ELSE 0 END)/count(SubscriptionID) as ProportionPaidSub,avg(NbrMealsPrice) as AvgPricePerMeal, sum(ProductDiscount) as TotalProductDiscount, sum(FormulaDiscount) as TotalFormulaDiscount, sum(TotalDiscount) as TotalDiscount, sum(TotalPrice) as TotalPrice, sum(TotalCredit) as TotalCredit,sum(SubscriptionDuration) as NbrDaysSub, avg(SubscriptionDuration) as AvgDurationPerSub, avg(NbrMealsPerDay) as AverageNbrMealPerDay, SUM(CASE WHEN ProductName='Custom Events' THEN 1 ELSE 0 END) as NbrCustomEventsProduct, SUM(CASE WHEN ProductName!='Custom Events' THEN 1 ELSE 0 END) as NbrGrubProduct from subscriptions group by CustomerID")
SubInter = SubInter.withColumn("FirstSubDate", SubInter.FirstSubDate*86400)
SubInter = SubInter.withColumn("FirstSubDate", SubInter.FirstSubDate.cast("timestamp"))
SubInter = SubInter.withColumn("EndOfLastSub", SubInter.EndOfLastSub*86400)
SubInter = SubInter.withColumn("EndOfLastSub", SubInter.EndOfLastSub.cast("timestamp"))
#SubInter = SubInter.withColumn("FirstSubDate", SubInter.select((unix_timestamp("FirstSubDate","yyy/MM/dd HH:mm:ss")).cast("timestamp")))
#df.select((unix_timestamp($"Date", "MM/dd/yyyy HH:mm:ss") * 1000).cast("timestamp"), $"Date")

#creating the churn dependent variable
SubInter = SubInter.withColumn("ChurnedAt03/02/2019", when(col("EndOfLastSub") > "2019-02-02 00:00:00", 0).otherwise(1))
SubInter = SubInter.withColumn("ChurnedAt03/08/2018", when(col("EndOfLastSub") > "2018-08-02 00:00:00", 0).otherwise(1))
SubInter = SubInter.withColumn("ChurnedAt03/02/2018", when(col("EndOfLastSub") > "2018-02-02 00:00:00", 0).otherwise(1))

display(SubInter)

CustomerID,TotalMeal_REG,MeanMeal_REGPerSub,TotalMeal_EXCEP,MeanMeal_EXCEPPerSub,FirstSubDate,EndOfLastSub,HasBeenClientForXDays,NbrSub,SubPaid,SubNotPaid,ProportionPaidSub,AvgPricePerMeal,TotalProductDiscount,TotalFormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,NbrDaysSub,AvgDurationPerSub,AverageNbrMealPerDay,NbrCustomEventsProduct,NbrGrubProduct,ChurnedAt03/02/2019,ChurnedAt03/08/2018,ChurnedAt03/02/2018
258487,785,196.25,55,13.75,2016-05-02T00:00:00.000+0000,2018-08-31T00:00:00.000+0000,851.0,4,3,1,0.75,17.216646666666666,0.0,409.6,409.6,11640.0,0.0,849.0,212.25,19.628179522497707,4,0,1,0,0
671995,1254,25.08,677,13.54,2015-01-02T00:00:00.000+0000,2019-02-14T00:00:00.000+0000,1504.0,50,50,0,1.0,20.5868036,0.0,0.0,0.0,25800.0,0.0,1452.0,29.04,0.8644984727298408,0,50,0,0,0
285977,1519,303.8,70,14.0,2014-01-04T00:00:00.000+0000,2019-02-07T00:00:00.000+0000,1860.0,5,5,0,1.0,15.705696,0.0,0.0,0.0,23858.199999999997,-721.8000000000001,1856.0,371.2,0.818706760989113,0,5,0,0,0
682942,1519,303.8,85,17.0,2014-02-01T00:00:00.000+0000,2019-01-31T00:00:00.000+0000,1825.0,5,5,0,1.0,16.180996,0.0,0.0,0.0,24580.0,0.0,1821.0,364.2,0.834157760048171,0,5,1,0,0
104880,1216,76.0,235,14.6875,2015-01-02T00:00:00.000+0000,2018-12-30T00:00:00.000+0000,1458.0,16,16,0,1.0,19.6064425,0.0,0.0,0.0,23840.0,0.0,1441.0,90.0625,0.8439920836606536,0,16,1,0,0
965578,1216,304.0,55,13.75,2014-12-19T00:00:00.000+0000,2018-12-17T00:00:00.000+0000,1459.0,4,4,0,1.0,15.62171,0.0,386.6,386.6,18996.0,0.0,1456.0,364.0,0.8351648351648352,0,4,1,0,0
829912,684,114.0,65,10.833333333333334,2016-03-19T00:00:00.000+0000,2017-12-31T00:00:00.000+0000,652.0,6,5,1,0.8333333333333334,14.713593333333334,0.0,787.11578,787.11578,9108.8,0.0,647.0,107.83333333333333,2.812704198909744,0,6,1,1,1
673836,1216,304.0,55,13.75,2015-01-02T00:00:00.000+0000,2019-01-03T00:00:00.000+0000,1462.0,4,4,0,1.0,16.41447,0.0,0.0,0.0,19960.0,-40.0,1459.0,364.75,0.8334705478974254,3,1,1,0,0
659301,1218,304.5,60,15.0,2014-06-23T00:00:00.000+0000,2018-06-24T00:00:00.000+0000,1462.0,4,4,0,1.0,15.961825000000001,0.0,0.0,0.0,19440.0,0.0,1458.0,364.5,0.8353925184404636,0,4,1,1,0
1012153,304,304.0,25,25.0,2017-03-02T00:00:00.000+0000,2018-03-01T00:00:00.000+0000,364.0,1,1,0,1.0,0.0,5180.0,0.0,5180.0,0.0,0.0,364.0,364.0,0.8351648351648352,0,1,1,1,0


In [8]:
Complaints.createOrReplaceTempView("complaints")

In [9]:
Intermediary = spark.sql("select CustomerID, count(ComplaintID) as NbrComplaints, max(ComplaintDate) as LastComplaint, min(ComplaintDate) as FirstComplaint, (CASE WHEN count(ComplaintID)>1 THEN (count(ComplaintID)/(max(ComplaintDate)-min(ComplaintDate))) ELSE 0 END) as ComplaintsPerMonth, SUM(CASE WHEN ProductID=1 THEN 1 ELSE 0 END) as NbrComplaintsProduct1, SUM(CASE WHEN ProductID=2 THEN 1 ELSE 0 END) as NbrComplaintsProduct2, SUM(CASE WHEN ProductID=3 THEN 1 ELSE 0 END) as NbrComplaintsProduct3, SUM(CASE WHEN ProductID=4 THEN 1 ELSE 0 END) as NbrComplaintsProduct4, SUM(CASE WHEN ProductID=5 THEN 1 ELSE 0 END) as NbrComplaintsProduct5, SUM(CASE WHEN ProductID=6 THEN 1 ELSE 0 END) as NbrComplaintsProduct6, SUM(CASE WHEN ProductID=7 THEN 1 ELSE 0 END) as NbrComplaintsProduct7 , SUM(CASE WHEN ProductID=8 THEN 1 ELSE 0 END) as NbrComplaintsProduct8, SUM(CASE WHEN ProductID=0 THEN 1 ELSE 0 END) as NbrComplaintsProductUnknown,SUM(CASE WHEN ComplaintTypeID=1 THEN 1 ELSE 0 END) as NbrComplaintsType1, SUM(CASE WHEN ComplaintTypeID=2 THEN 1 ELSE 0 END) as NbrComplaintsType2, SUM(CASE WHEN ComplaintTypeID=3 THEN 1 ELSE 0 END) as NbrComplaintsType3, SUM(CASE WHEN ComplaintTypeID=4 THEN 1 ELSE 0 END) as NbrComplaintsType4, SUM(CASE WHEN ComplaintTypeID=5 THEN 1 ELSE 0 END) as NbrComplaintsType5, SUM(CASE WHEN ComplaintTypeID=6 THEN 1 ELSE 0 END) as NbrComplaintsType6, SUM(CASE WHEN ComplaintTypeID=7 THEN 1 ELSE 0 END) as NbrComplaintsType7 , SUM(CASE WHEN ComplaintTypeID=8 THEN 1 ELSE 0 END) as NbrComplaintsType8, SUM(CASE WHEN ComplaintTypeID=9 THEN 1 ELSE 0 END) as NbrComplaintsType9, SUM(CASE WHEN ComplaintTypeID=0 THEN 1 ELSE 0 END) as NbrComplaintsTypeUnknown, SUM(CASE WHEN SolutionTypeID=1 THEN 1 ELSE 0 END) as NbrSolutionsType1, SUM(CASE WHEN SolutionTypeID=2 THEN 1 ELSE 0 END) as NbrSolutionsType2, SUM(CASE WHEN SolutionTypeID=3 THEN 1 ELSE 0 END) as NbrSolutionsType3, SUM(CASE WHEN SolutionTypeID=4 THEN 1 ELSE 0 END) as NbrSolutionsType4, SUM(CASE WHEN SolutionTypeID=0 THEN 1 ELSE 0 END) as NbrSolutionsTypeUnknown from complaints group by CustomerID")
Intermediary = Intermediary.withColumn("FirstComplaint", Intermediary.FirstComplaint*86400)
Intermediary = Intermediary.withColumn("FirstComplaint", Intermediary.FirstComplaint.cast("timestamp"))
Intermediary = Intermediary.withColumn("LastComplaint", Intermediary.LastComplaint*86400)
Intermediary = Intermediary.withColumn("LastComplaint", Intermediary.LastComplaint.cast("timestamp"))

display(Intermediary)

CustomerID,NbrComplaints,LastComplaint,FirstComplaint,ComplaintsPerMonth,NbrComplaintsProduct1,NbrComplaintsProduct2,NbrComplaintsProduct3,NbrComplaintsProduct4,NbrComplaintsProduct5,NbrComplaintsProduct6,NbrComplaintsProduct7,NbrComplaintsProduct8,NbrComplaintsProductUnknown,NbrComplaintsType1,NbrComplaintsType2,NbrComplaintsType3,NbrComplaintsType4,NbrComplaintsType5,NbrComplaintsType6,NbrComplaintsType7,NbrComplaintsType8,NbrComplaintsType9,NbrComplaintsTypeUnknown,NbrSolutionsType1,NbrSolutionsType2,NbrSolutionsType3,NbrSolutionsType4,NbrSolutionsTypeUnknown
285977,7,2018-11-27T00:00:00.000+0000,2014-09-18T00:00:00.000+0000,0.0045721750489875895,0,0,0,0,0,7,0,0,0,4,1,1,0,0,0,0,0,1,0,1,0,1,0,5
671995,2,2014-10-14T00:00:00.000+0000,2014-03-25T00:00:00.000+0000,0.009852216748768473,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2
466728,1,2016-12-15T00:00:00.000+0000,2016-12-15T00:00:00.000+0000,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1012153,3,2018-08-07T00:00:00.000+0000,2018-08-02T00:00:00.000+0000,0.6,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1,1,1,0,0
673836,29,2018-12-24T00:00:00.000+0000,2013-11-14T00:00:00.000+0000,0.015541264737406217,0,0,0,21,0,0,0,0,8,18,2,0,0,7,0,1,0,1,0,2,2,4,0,21
865501,1,2016-02-04T00:00:00.000+0000,2016-02-04T00:00:00.000+0000,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
802664,1,2013-06-21T00:00:00.000+0000,2013-06-21T00:00:00.000+0000,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
204587,6,2017-02-19T00:00:00.000+0000,2015-12-14T00:00:00.000+0000,0.013856812933025405,0,0,0,0,0,0,0,6,0,6,0,0,0,0,0,0,0,0,0,6,0,0,0,0
462878,2,2017-03-02T00:00:00.000+0000,2012-04-08T00:00:00.000+0000,0.0011179429849077697,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1
67278,2,2018-11-08T00:00:00.000+0000,2018-07-19T00:00:00.000+0000,0.017857142857142856,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0


In [10]:
display(SubInter)

CustomerID,TotalMeal_REG,MeanMeal_REGPerSub,TotalMeal_EXCEP,MeanMeal_EXCEPPerSub,FirstSubDate,EndOfLastSub,HasBeenClientForXDays,NbrSub,SubPaid,SubNotPaid,ProportionPaidSub,AvgPricePerMeal,TotalProductDiscount,TotalFormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,NbrDaysSub,AvgDurationPerSub,AverageNbrMealPerDay,NbrCustomEventsProduct,NbrGrubProduct,ChurnedAt03/02/2019,ChurnedAt03/08/2018,ChurnedAt03/02/2018
258487,785,196.25,55,13.75,2016-05-02T00:00:00.000+0000,2018-08-31T00:00:00.000+0000,851.0,4,3,1,0.75,17.216646666666666,0.0,409.6,409.6,11640.0,0.0,849.0,212.25,19.628179522497707,4,0,1,0,0
671995,1254,25.08,677,13.54,2015-01-02T00:00:00.000+0000,2019-02-14T00:00:00.000+0000,1504.0,50,50,0,1.0,20.5868036,0.0,0.0,0.0,25800.0,0.0,1452.0,29.04,0.8644984727298408,0,50,0,0,0
285977,1519,303.8,70,14.0,2014-01-04T00:00:00.000+0000,2019-02-07T00:00:00.000+0000,1860.0,5,5,0,1.0,15.705696,0.0,0.0,0.0,23858.199999999997,-721.8000000000001,1856.0,371.2,0.818706760989113,0,5,0,0,0
682942,1519,303.8,85,17.0,2014-02-01T00:00:00.000+0000,2019-01-31T00:00:00.000+0000,1825.0,5,5,0,1.0,16.180996,0.0,0.0,0.0,24580.0,0.0,1821.0,364.2,0.834157760048171,0,5,1,0,0
104880,1216,76.0,235,14.6875,2015-01-02T00:00:00.000+0000,2018-12-30T00:00:00.000+0000,1458.0,16,16,0,1.0,19.6064425,0.0,0.0,0.0,23840.0,0.0,1441.0,90.0625,0.8439920836606536,0,16,1,0,0
965578,1216,304.0,55,13.75,2014-12-19T00:00:00.000+0000,2018-12-17T00:00:00.000+0000,1459.0,4,4,0,1.0,15.62171,0.0,386.6,386.6,18996.0,0.0,1456.0,364.0,0.8351648351648352,0,4,1,0,0
829912,684,114.0,65,10.833333333333334,2016-03-19T00:00:00.000+0000,2017-12-31T00:00:00.000+0000,652.0,6,5,1,0.8333333333333334,14.713593333333334,0.0,787.11578,787.11578,9108.8,0.0,647.0,107.83333333333333,2.812704198909744,0,6,1,1,1
673836,1216,304.0,55,13.75,2015-01-02T00:00:00.000+0000,2019-01-03T00:00:00.000+0000,1462.0,4,4,0,1.0,16.41447,0.0,0.0,0.0,19960.0,-40.0,1459.0,364.75,0.8334705478974254,3,1,1,0,0
659301,1218,304.5,60,15.0,2014-06-23T00:00:00.000+0000,2018-06-24T00:00:00.000+0000,1462.0,4,4,0,1.0,15.961825000000001,0.0,0.0,0.0,19440.0,0.0,1458.0,364.5,0.8353925184404636,0,4,1,1,0
1012153,304,304.0,25,25.0,2017-03-02T00:00:00.000+0000,2018-03-01T00:00:00.000+0000,364.0,1,1,0,1.0,0.0,5180.0,0.0,5180.0,0.0,0.0,364.0,364.0,0.8351648351648352,0,1,1,1,0


In [11]:
#Base Table
#base = Customers.join(Complaints,on=['CustomerID'],how='full')
#base = Customers.join(Subscriptions,on=['CustomerID'],how='full')
base = Customers.join(Intermediary,on=['CustomerID'],how='full')
base1 = base.join(SubInter,on=['CustomerID'],how='full')
#Replacing null in Complaints
base1 = base1.na.fill(0)

In [12]:
display(base1)

CustomerID,Region,StreetID,NbrComplaints,LastComplaint,FirstComplaint,ComplaintsPerMonth,NbrComplaintsProduct1,NbrComplaintsProduct2,NbrComplaintsProduct3,NbrComplaintsProduct4,NbrComplaintsProduct5,NbrComplaintsProduct6,NbrComplaintsProduct7,NbrComplaintsProduct8,NbrComplaintsProductUnknown,NbrComplaintsType1,NbrComplaintsType2,NbrComplaintsType3,NbrComplaintsType4,NbrComplaintsType5,NbrComplaintsType6,NbrComplaintsType7,NbrComplaintsType8,NbrComplaintsType9,NbrComplaintsTypeUnknown,NbrSolutionsType1,NbrSolutionsType2,NbrSolutionsType3,NbrSolutionsType4,NbrSolutionsTypeUnknown,TotalMeal_REG,MeanMeal_REGPerSub,TotalMeal_EXCEP,MeanMeal_EXCEPPerSub,FirstSubDate,EndOfLastSub,HasBeenClientForXDays,NbrSub,SubPaid,SubNotPaid,ProportionPaidSub,AvgPricePerMeal,TotalProductDiscount,TotalFormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,NbrDaysSub,AvgDurationPerSub,AverageNbrMealPerDay,NbrCustomEventsProduct,NbrGrubProduct,ChurnedAt03/02/2019,ChurnedAt03/08/2018,ChurnedAt03/02/2018
104880,5,45805,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1216,76.0,235,14.6875,2015-01-02T00:00:00.000+0000,2018-12-30T00:00:00.000+0000,1458.0,16,16,0,1.0,19.6064425,0.0,0.0,0.0,23840.0,0.0,1441.0,90.0625,0.8439920836606536,0,16,1,0,0
258487,1,14628,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,785,196.25,55,13.75,2016-05-02T00:00:00.000+0000,2018-08-31T00:00:00.000+0000,851.0,4,3,1,0.75,17.216646666666666,0.0,409.6,409.6,11640.0,0.0,849.0,212.25,19.628179522497707,4,0,1,0,0
285977,1,18415,7,2018-11-27T00:00:00.000+0000,2014-09-18T00:00:00.000+0000,0.0045721750489875895,0,0,0,0,0,7,0,0,0,4,1,1,0,0,0,0,0,1,0,1,0,1,0,5,1519,303.8,70,14.0,2014-01-04T00:00:00.000+0000,2019-02-07T00:00:00.000+0000,1860.0,5,5,0,1.0,15.705696,0.0,0.0,0.0,23858.199999999997,-721.8000000000001,1856.0,371.2,0.818706760989113,0,5,0,0,0
671995,1,28929,2,2014-10-14T00:00:00.000+0000,2014-03-25T00:00:00.000+0000,0.009852216748768473,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1254,25.08,677,13.54,2015-01-02T00:00:00.000+0000,2019-02-14T00:00:00.000+0000,1504.0,50,50,0,1.0,20.5868036,0.0,0.0,0.0,25800.0,0.0,1452.0,29.04,0.8644984727298408,0,50,0,0,0
682942,1,18048,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1519,303.8,85,17.0,2014-02-01T00:00:00.000+0000,2019-01-31T00:00:00.000+0000,1825.0,5,5,0,1.0,16.180996,0.0,0.0,0.0,24580.0,0.0,1821.0,364.2,0.834157760048171,0,5,1,0,0
829912,5,40317,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,684,114.0,65,10.833333333333334,2016-03-19T00:00:00.000+0000,2017-12-31T00:00:00.000+0000,652.0,6,5,1,0.8333333333333334,14.713593333333334,0.0,787.11578,787.11578,9108.8,0.0,647.0,107.83333333333333,2.812704198909744,0,6,1,1,1
965578,5,45860,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1216,304.0,55,13.75,2014-12-19T00:00:00.000+0000,2018-12-17T00:00:00.000+0000,1459.0,4,4,0,1.0,15.62171,0.0,386.6,386.6,18996.0,0.0,1456.0,364.0,0.8351648351648352,0,4,1,0,0
75070,5,43993,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,912,304.0,45,15.0,2015-01-02T00:00:00.000+0000,2017-06-29T00:00:00.000+0000,909.0,3,3,0,1.0,16.29386,0.0,0.0,0.0,14860.0,0.0,907.0,302.3333333333333,1.1205064295973386,0,3,1,1,1
107896,5,46836,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1216,304.0,55,13.75,2015-01-02T00:00:00.000+0000,2018-12-30T00:00:00.000+0000,1458.0,4,4,0,1.0,16.61184,0.0,0.0,0.0,20200.0,0.0,1455.0,363.75,0.8357400175581995,0,4,1,0,0
158050,5,41138,0,null,null,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1216,304.0,55,13.75,2015-01-02T00:00:00.000+0000,2018-01-20T00:00:00.000+0000,1114.0,4,3,1,0.75,16.575655,0.0,0.0,0.0,14816.0,-44.0,1111.0,277.75,4.426948808766991,0,4,1,1,1


In [13]:
#Create categorical variables for Region and StreetID 
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer
from pyspark.ml import Pipeline

#Region
genderIndxr = StringIndexer().setInputCol("Region").setOutputCol("RegionInd")

#StreetID
classIndxr = StringIndexer().setInputCol("StreetID").setOutputCol("StreetIDInd")

#One-hot encoding
ohee_catv = OneHotEncoderEstimator(inputCols=["RegionInd","StreetIDInd"],outputCols=["Region_dum","StreetID_dum"])
pipe_catv = Pipeline(stages=[genderIndxr, classIndxr, ohee_catv])

basetable_final = pipe_catv.fit(base1).transform(base1)
basetable_final=basetable_final.drop("RegionInd","StreetIDInd","EndOfLastSub","LastComplaint","FirstComplaint","ChurnedAt03/02/2018","ChurnedAt03/02/2019")


basetable_final= basetable_final.withColumnRenamed("ChurnedAt03/08/2018","label")

In [14]:

#Splitting the data set into train and test
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window

basetable_final = basetable_final.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("FirstSubDate")))

In [15]:
basetable_train = basetable_final.where("rank <= .736").drop("rank","FirstSubDate")
basetable_test = basetable_final.where("rank > .736").drop("rank","FirstSubDate")

In [16]:
basetable_train.where((col("label") == 1)).count()

Out[32]: 353

In [17]:
#Transform the tables in a table of label, features format
from pyspark.ml.feature import RFormula

trainBig = RFormula(formula="label ~ . - CustomerID").fit(basetable_final).transform(basetable_final)
train = RFormula(formula="label ~ . - CustomerID").fit(basetable_train).transform(basetable_train)
test = RFormula(formula="label ~ . - CustomerID").fit(basetable_test).transform(basetable_test)
print("trainBig nobs: " + str(trainBig.count()))
print("train nobs: " + str(train.count()))
print("test nobs: " + str(test.count()))

trainBig nobs: 1389
train nobs: 1023
test nobs: 366

In [18]:
#Train a Logistic Regression model
from pyspark.ml.classification import LogisticRegression

#Define the algorithm class
lr = LogisticRegression()

#Fit the model
lrModel = lr.fit(trainBig)

#Print coefficients
lrModel.coefficients

Out[34]: DenseVector([125.7169, 0.0301, -23.8086, 206.4476, 1813.9119, 40.9338, -592.6452, 104.4841, -78.1597, -354.4454, -1284.2058, 33.356, -199.2873, 9.0762, -436.0739, -738.6205, 615.2262, -541.2777, 2389.2046, 227.4351, -2916.9645, -725.8547, 0.0, -77.9915, 120.617, -382.8711, -125.2936, -27.1199, -0.3349, 4.6481, -2.7114, -37.2839, -2.5237, -48.8272, -64.0657, 1721.7272, 2504.3971, 81.4347, -0.0961, -0.5766, -0.1496, -0.0701, -0.4268, -0.9643, 1.6245, -11.5147, -92.1809, -44.816, 59.1058, 983.1677, -750.8568, 1142.6274, 4110.7499, 470.6377, 413.7174, 3629.0042, 2199.3158, 1513.2089, 1493.5148, 2720.6202, 3605.123, 6640.7965, 719.6184, -4092.1675, 810.3517, 1625.32, 1341.3008, -6032.1315, 6321.1376, -3163.2702, 1089.7485, 5802.9947, -1636.8684, 5882.356, 4463.9749, 6240.6441, 6802.9341, 1423.4165, 1150.717, -3025.1516, 2133.7451, 1475.345, 1584.4149, 1141.8164, 1332.0992, -467.4915, 3415.2555, 797.2492, 2792.0668, 1292.8076, -3791.5026, 4434.3883, 4654.0385, 4797.7088, 1391.9915, 1571.3159, 951.6311, -3146.0806, 609.818, 1428.3077, -5704.7233, -3850.9117, 3699.6254, 6253.9437, 6278.2467, 3276.4678, -3540.2134, -4363.1954, -4009.7429, 5060.6113, 3686.2674, -3714.164, 1539.4355, 526.9404, 6651.0426, -3938.0188, -2798.5589, -1614.8877, 5072.4963, -3722.1994, 6664.6781, -4037.6261, 3384.868, 3578.4756, 5021.312, -1465.2504, 3594.9636, -4060.5225, -2649.4765, -4381.8003, -2536.1161, 3909.5274, 3445.7163, 6398.5077, -4244.008, 3438.5319, -5737.5551, -1717.236, 3316.323, -4800.336, -3035.4277, -1304.8668, 5323.4596, 4176.5086, 3475.7158, -1485.9995, 7382.2007, -4040.5912, 6730.9363, -1363.2514, -7199.6371, 3407.229, 6284.2499, -3635.6436, 5385.9616, -4079.7793, -5759.7133, -2415.2586, 6230.4489, 5523.9343, -3542.7959, -3733.7844, 4112.2254, -2029.4382, 6387.4431, 5822.7616, 3188.66, 3429.1976, 4230.047, -2744.0997, 4170.0509, -4447.7213, -3708.7303, -4463.7973, 6851.3547, -3767.4063, 3415.3401, 3443.0815, -3949.0978, -3797.7363, -7308.9574, -1657.7114, -7561.8116, 6710.8403, 8165.6471, 3363.8809, -2577.0366, 3732.1174, -1631.0617, 3928.7186, -4007.3416, -4136.9611, -3815.2193, 5424.6063, 3453.5406, 6797.3776, -5847.504, 7347.9951, -3883.779, -3402.5215, -1367.8744, 3353.3092, -4566.596, -2524.5058, 3321.9186, 3329.1653, 6628.9273, -3524.3616, -4050.5953, 4776.8078, 3287.8634, 6880.8557, -1755.9762, -4505.1565, -3726.5413, -5494.2875, -5065.7374, 3879.3214, -3831.9022, 3289.5842, -4649.4103, -2833.0927, 3519.0311, -2676.5744, -3793.2545, 5054.4382, 4987.1157, -6370.7918, -2815.7882, 5854.9338, -3218.5232, 3398.5756, 5025.0034, 7104.588, -2627.7447, -5779.8222, -5510.7526, -3831.5987, -4530.4796, 3387.1344, -2583.668, -3972.5309, 3428.7479, -3895.6555, 6483.1144, -6841.7128, 6684.4292, 6866.4828, 3427.4613, -4112.737, 6815.3886, -3907.4135, 3348.2958, 6687.883, 3489.4657, -2686.2382, 6780.2684, 3403.5766, 3850.0322, -2636.8641, 3279.4719, 6718.6128, -1786.2953, -3761.7925, -1656.1905, -3702.3646, -4149.1971, -1361.1848, -2649.2765, 3473.5349, 3251.1526, -3313.9448, -4370.4862, -2584.2291, 4427.8849, -3701.1813, -2384.4751, -7954.302, -2604.6166, 6645.1184, 5259.4335, -3862.2416, -1260.6804, -9681.0691, -4166.1589, 5601.3076, -2691.9376, -5205.5477, -5495.4266, -4736.1453, 3502.4111, -2434.6892, -3652.1731, 6073.5084, 3494.6859, -3811.1793, -2201.9171, -1526.8698, 3624.0084, -2825.3209, 5192.0904, -7807.1445, -4718.5039, 3442.9458, 3958.0222, 6404.8048, -2026.4079, 6970.2097, 5311.9111, -4071.4264, -3648.2354, 3378.3362, 8190.6374, 6618.1728, -7527.0758, -3813.7837, 5374.2571, -2497.88, -3770.5583, 3843.0714, -3661.0672, 3411.2661, -2787.772, 8650.6665, -4067.0475, -2677.8082, 5160.7379, -3759.5477, -3706.7341, 3683.0723, 5277.9175, 3357.4011, 6917.4491, -4538.5251, -2185.551, -1853.179, 3611.4352, 3310.9176, -3754.0279, -3695.5791, 3498.7866, -4264.8124, 3480.6631, -3832.2666, -3574.1204, -5762.7398, -3873.6381, 6192.7898, 6776.9535, -3402.509, 6822.5976, -9326.3673, -2073.7104, 3345.3058, -3777.073, -3876.9365, -1829.7761, 6522.6

In [19]:
#Spark offers two options for performing hyperparameter tuning automatically:

#1. TrainValidationSplit: randomly split data in 2 groups
from pyspark.ml.tuning import TrainValidationSplit

#2. CrossValidator: k-fold cross-validation by splitting the data into k non-overlapping, randomly partitioned folds
from pyspark.ml.tuning import CrossValidator

#First method is good for quick model evaluations, 2nd method is recommended for a more rigorous model evaluation.


#Hyperparameter tuning for different hyperparameter values of LR (aka model selection)
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#Define pipeline
lr = LogisticRegression()
pipeline = Pipeline().setStages([lr])

#Set param grid
lrparams = ParamGridBuilder()\
  .addGrid(lr.regParam, [0.1, 0.01])\
  .addGrid(lr.maxIter, [50, 100,150])\
  .build()

#Evaluator: uses max(AUC) by default to get the final model
evaluator = BinaryClassificationEvaluator()
#Check params through: evaluator.explainParams()

#Cross-validation of entire pipeline
cv = CrossValidator()\
  .setEstimator(pipeline)\
  .setEstimatorParamMaps(lrparams)\
  .setEvaluator(evaluator)\
  .setNumFolds(5) # Here: 5-fold cross validation

#Run cross-validation, and choose the best set of parameters.
#Spark automatically saves the best model in cvModel.
cvModel = cv.fit(train)


#Get best tuned parameters of pipeline
cvBestPipeline = cvModel.bestModel
cvBestLRModel = cvBestPipeline.stages[-1]._java_obj.parent() #the stages function refers to the stage in the pipelinemodel

print("Best LR model:")
print("** regParam: " + str(cvBestLRModel.getRegParam()))
print("** maxIter: " + str(cvBestLRModel.getMaxIter()))

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
Best LR model:
** regParam: 0.1
** maxIter: 50

In [20]:
preds = cvModel.transform(test)\
  .select("prediction", "label")
preds.show(10)

#Get model performance on test set
from pyspark.mllib.evaluation import BinaryClassificationMetrics

out = preds.rdd.map(lambda x: (float(x[0]), float(x[1])))
metrics = BinaryClassificationMetrics(out)

print(metrics.areaUnderPR) #area under precision/recall curve
print(metrics.areaUnderROC)#area under Receiver Operating Characteristic curve

+----------+-----+
prediction|label|
+----------+-----+
 1.0| 1|
 1.0| 0|
 1.0| 0|
 1.0| 1|
 1.0| 0|
 1.0| 1|
 1.0| 0|
 1.0| 1|
 1.0| 1|
 0.0| 0|
+----------+-----+
only showing top 10 rows

0.75
0.5686274509803921

In [21]:
#Exercise: Train a RandomForest model and tune the number of trees for values [150, 300, 500]
#Hint: analogous to buidling a LR model (see above)
from pyspark.ml.classification import RandomForestClassifier

#Define pipeline
rfc = RandomForestClassifier()
rfPipe = Pipeline().setStages([rfc])

#Set param grid
rfParams = ParamGridBuilder()\
  .addGrid(rfc.numTrees, [150, 300, 500])\
  .build()

rfCv = CrossValidator()\
  .setEstimator(rfPipe)\
  .setEstimatorParamMaps(rfParams)\
  .setEvaluator(BinaryClassificationEvaluator())\
  .setNumFolds(5) # Here: 5-fold cross validation

#Run cross-validation, and choose the best set of parameters.
rfcModel = rfCv.fit(train)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [22]:
#Get predictions on the test set
preds = rfcModel.transform(test)
display(preds)

CustomerID,Region,StreetID,NbrComplaints,ComplaintsPerMonth,NbrComplaintsProduct1,NbrComplaintsProduct2,NbrComplaintsProduct3,NbrComplaintsProduct4,NbrComplaintsProduct5,NbrComplaintsProduct6,NbrComplaintsProduct7,NbrComplaintsProduct8,NbrComplaintsProductUnknown,NbrComplaintsType1,NbrComplaintsType2,NbrComplaintsType3,NbrComplaintsType4,NbrComplaintsType5,NbrComplaintsType6,NbrComplaintsType7,NbrComplaintsType8,NbrComplaintsType9,NbrComplaintsTypeUnknown,NbrSolutionsType1,NbrSolutionsType2,NbrSolutionsType3,NbrSolutionsType4,NbrSolutionsTypeUnknown,TotalMeal_REG,MeanMeal_REGPerSub,TotalMeal_EXCEP,MeanMeal_EXCEPPerSub,HasBeenClientForXDays,NbrSub,SubPaid,SubNotPaid,ProportionPaidSub,AvgPricePerMeal,TotalProductDiscount,TotalFormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,NbrDaysSub,AvgDurationPerSub,AverageNbrMealPerDay,NbrCustomEventsProduct,NbrGrubProduct,label,Region_dum,StreetID_dum,features,rawPrediction,probability,prediction
56451,1,14405,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,12.0,0,0.0,286.0,2,2,0,1.0,0.0,0.0,499.2,499.2,0.0,0.0,26.0,13.0,0.9230769230769231,1,1,1,"List(0, 8, List(1), List(1.0))","List(0, 1282, List(798), List(1.0))","List(0, 1338, List(0, 1, 28, 29, 32, 33, 34, 36, 39, 40, 43, 44, 45, 46, 47, 49, 854), List(1.0, 14405.0, 24.0, 12.0, 286.0, 2.0, 2.0, 1.0, 499.2, 499.2, 26.0, 13.0, 0.9230769230769231, 1.0, 1.0, 1.0, 1.0))","List(1, 2, List(), List(178.14628712310395, 321.85371287689657))","List(1, 2, List(), List(0.3562925742462075, 0.6437074257537925))",1.0
709042,1,16777,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,49,16.333333333333332,25,8.333333333333334,1128.0,3,3,0,1.0,3.733333333333333,0.0,779.2,779.2,280.0,0.0,53.0,17.666666666666668,0.9386446886446885,0,3,0,"List(0, 8, List(1), List(1.0))","List(0, 1282, List(31), List(1.0))","List(0, 1338, List(0, 1, 28, 29, 30, 31, 32, 33, 34, 36, 37, 39, 40, 41, 43, 44, 45, 47, 49, 87), List(1.0, 16777.0, 49.0, 16.333333333333332, 25.0, 8.333333333333334, 1128.0, 3.0, 3.0, 1.0, 3.733333333333333, 779.2, 779.2, 280.0, 53.0, 17.666666666666668, 0.9386446886446885, 3.0, 1.0, 1.0))","List(1, 2, List(), List(175.75923446110272, 324.2407655388978))","List(1, 2, List(), List(0.35151846892220506, 0.6484815310777949))",1.0
60536,1,19113,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,912,304.0,45,15.0,1093.0,3,3,0,1.0,16.776313333333334,0.0,0.0,0.0,15300.0,0.0,1091.0,363.6666666666667,0.8359317450226542,0,3,0,"List(0, 8, List(1), List(1.0))","List(0, 1282, List(339), List(1.0))","List(0, 1338, List(0, 1, 28, 29, 30, 31, 32, 33, 34, 36, 37, 41, 43, 44, 45, 47, 49, 395), List(1.0, 19113.0, 912.0, 304.0, 45.0, 15.0, 1093.0, 3.0, 3.0, 1.0, 16.776313333333334, 15300.0, 1091.0, 363.6666666666667, 0.8359317450226542, 3.0, 1.0, 1.0))","List(1, 2, List(), List(165.88437856286737, 334.1156214371329))","List(1, 2, List(), List(0.33176875712573456, 0.6682312428742655))",1.0
61494,7,69155,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,734,183.5,55,13.75,678.0,4,3,1,0.75,14.873686666666666,0.0,413.6,413.6,10720.0,0.0,771.0,192.75,0.9292888097931706,0,4,1,"List(0, 8, List(2), List(1.0))","List(0, 1282, List(75), List(1.0))","List(0, 1338, List(0, 1, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 47, 50, 131), List(7.0, 69155.0, 734.0, 183.5, 55.0, 13.75, 678.0, 4.0, 3.0, 1.0, 0.75, 14.873686666666666, 413.6, 413.6, 10720.0, 771.0, 192.75, 0.9292888097931706, 4.0, 1.0, 1.0))","List(1, 2, List(), List(195.34113967334736, 304.6588603266524))","List(1, 2, List(), List(0.3906822793466949, 0.6093177206533051))",1.0
55101,1,18256,1,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1216,304.0,55,13.75,1107.0,4,3,1,0.75,17.12171,0.0,0.0,0.0,15300.0,0.0,1104.0,276.0,6.959710111391306,0,4,0,"List(0, 8, List(1), List(1.0))","List(0, 1282, List(1069), List(1.0))","List(0, 1338, List(0, 1, 2, 9, 21, 23, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 41, 43, 44, 45, 47, 49, 1125), List(1.0, 18256.0, 1.0, 1.0, 1.0, 1.0, 1216.0, 304.0, 55.0,

In [23]:
#Get model accuracy
print("accuracy: " + str(evaluator.evaluate(preds)))

#Get AUC
metrics = BinaryClassificationMetrics(preds.rdd.map(lambda x: (float(x[0]), float(x[1]))))
print("AUC: " + str(metrics.areaUnderROC))

accuracy: 0.7655228758169934
AUC: 1.0

In [24]:
#Get more metrics
from pyspark.mllib.evaluation import MulticlassMetrics

#Cast a DF of predictions to an RDD to access RDD methods of MulticlassMetrics
preds_labels = cvModel.transform(test)\
  .select("prediction", "label")\
  .rdd.map(lambda x: (float(x[0]), float(x[1])))

metrics = MulticlassMetrics(preds_labels)

print("accuracy = %s" % metrics.accuracy)

accuracy = 0.7595628415300546

In [25]:
#Get more metrics
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.util import MLUtils

labels = preds.rdd.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
 #   print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

Class 0 precision = 1.0
Class 0 recall = 0.13725490196078433
Class 1 precision = 0.75
Class 1 recall = 1.0

In [26]:
#Select the best RF model
rfcBestModel = rfcModel.bestModel.stages[-1] #-1 means "get last stage in the pipeline"


In [27]:
#Get tuned number of trees
rfcBestModel.getNumTrees

Out[44]: 500

In [28]:
#Prettify feature importances
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))
  
ExtractFeatureImp(rfcBestModel.featureImportances, train, "features").head(10)

,idx,name,score
41,41,TotalPrice,0.117071
32,32,HasBeenClientForXDays,0.085446
43,43,NbrDaysSub,0.082592
31,31,MeanMeal_EXCEPPerSub,0.074673
34,34,SubPaid,0.060538
37,37,AvgPricePerMeal,0.059804
33,33,NbrSub,0.052510
28,28,TotalMeal_REG,0.050055
30,30,TotalMeal_EXCEP,0.047162
47,47,NbrGrubProduct,0.038351


In [29]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)
predictions.select( 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+-----+-------------+----------+--------------------+
label|rawPrediction|prediction| probability|
+-----+-------------+----------+--------------------+
 1| [23.0,194.0]| 1.0|[0.10599078341013...|
 0| [23.0,194.0]| 1.0|[0.10599078341013...|
 0| [23.0,194.0]| 1.0|[0.10599078341013...|
 1| [23.0,194.0]| 1.0|[0.10599078341013...|
 0| [23.0,194.0]| 1.0|[0.10599078341013...|
 1| [23.0,194.0]| 1.0|[0.10599078341013...|
 0| [23.0,194.0]| 1.0|[0.10599078341013...|
 1| [23.0,194.0]| 1.0|[0.10599078341013...|
 1| [23.0,194.0]| 1.0|[0.10599078341013...|
 0| [23.0,194.0]| 1.0|[0.10599078341013...|
+-----+-------------+----------+--------------------+
only showing top 10 rows

In [30]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.5

In [31]:
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier,RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder,TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#Define pipeline
lr = LogisticRegression()
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
rfc = RandomForestClassifier()

pipe = Pipeline().setStages([lr,rfc,dt])

In [32]:
#Set param grid of lr
params = ParamGridBuilder()\
  .addGrid(lr.regParam, [0.1, 0.01])\
  .addGrid(lr.maxIter, [50, 100,150])\
  .build()

#Evaluator: uses max(AUC) by default to get the final model
evaluator = BinaryClassificationEvaluator()
#Check params through: evaluator.explainParams()


#Set param grid
rfParams = ParamGridBuilder()\
  .addGrid(rfc.numTrees, [150, 300, 500])\
  .build()

In [33]:


#Cross-validation of entire pipeline
cv = CrossValidator()\
  .setEstimator(pipe)\
  .setEstimatorParamMaps(params)\
  .setEvaluator(evaluator)\
  .setNumFolds(5) # Here: 5-fold cross validation

#Run cross-validation, and choose the best set of parameters.
#Spark automatically saves the best model in cvModel.
cvModel = cv.fit(train)


#Get best tuned parameters of pipeline
cvBestPipeline = cvModel.bestModel
cvBestLRModel = cvBestPipeline.stages[-1]._java_obj.parent()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o13733.fit.
: java.lang.IllegalArgumentException: requirement failed: Column prediction already exists.
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.util.SchemaUtils$.appendColumn(SchemaUtils.scala:105)
	at org.apache.spark.ml.util.SchemaUtils$.appendColumn(SchemaUtils.scala:95)
	at org.apache.spark.ml.PredictorParams$class.validateAndTransformSchema(Predictor.scala:65)
	at org.apache.spark.ml.classification.Classifier.org$apache$spark$ml$classification$ClassifierParams$$super$validateAndTransformSchema(Classifier.scala:60)
	at org.apache.spark.ml.classification.ClassifierParams$class.validateAndTransformSchema(Classifier.scala:44)
	at org.apache.spark.ml.classification.ProbabilisticClassifier.org$apache$spark$ml$classification$ProbabilisticClassifierParams$$super$validateAndTransformSchema(ProbabilisticClassifier.scala:55)
	at org.apache.spark.ml.classification.ProbabilisticClassifierParams$class.validateAndTransformSchema(ProbabilisticClassifier.scala:39)
	at org.apache.spark.ml.classification.ProbabilisticClassifier.validateAndTransformSchema(ProbabilisticClassifier.scala:55)
	at org.apache.spark.ml.Predictor.transformSchema(Predictor.scala:146)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:74)
	at org.apache.spark.ml.Predictor.fit(Predictor.scala:102)
	at sun.reflect.GeneratedMethodAccessor613.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-3017412667564611> in <module> 
 10 #Run cross-validation, and choose the best set of parameters. 
 11 #Spark automatically saves the best model in cvModel. 
 ---> 12 cvModel = cv . fit ( train ) 
 13 
 14 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/tuning.py in _fit (self, dataset) 
 308 
 309 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 310 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 311 metrics [ j ] += ( metric / nFolds ) 
 312 metrics_all [ i ] [ j ] = metric

 /usr/lib/python3.7/multiprocessing/pool.py in next (self, timeout) 
 746 if success : 
 747 return value
 --> 748 raise value
 749 
 750 __next__ = next # XXX 

 /usr/lib/python3.7/multiprocessing/pool.py in worker (inqueue, outqueue, initializer, initargs, maxtasks, wrap_exception) 
 119 job , i , func , args , kwds = task
 120 try : 
 --> 121 result = ( True , func ( * args , ** kwds ) ) 
 122 except Exception as e : 
 123 if wrap_exception and func is not _helper_reraises_excep

In [34]:
display(train)

CustomerID,Region,StreetID,NbrComplaints,ComplaintsPerMonth,NbrComplaintsProduct1,NbrComplaintsProduct2,NbrComplaintsProduct3,NbrComplaintsProduct4,NbrComplaintsProduct5,NbrComplaintsProduct6,NbrComplaintsProduct7,NbrComplaintsProduct8,NbrComplaintsProductUnknown,NbrComplaintsType1,NbrComplaintsType2,NbrComplaintsType3,NbrComplaintsType4,NbrComplaintsType5,NbrComplaintsType6,NbrComplaintsType7,NbrComplaintsType8,NbrComplaintsType9,NbrComplaintsTypeUnknown,NbrSolutionsType1,NbrSolutionsType2,NbrSolutionsType3,NbrSolutionsType4,NbrSolutionsTypeUnknown,TotalMeal_REG,MeanMeal_REGPerSub,TotalMeal_EXCEP,MeanMeal_EXCEPPerSub,HasBeenClientForXDays,NbrSub,SubPaid,SubNotPaid,ProportionPaidSub,AvgPricePerMeal,TotalProductDiscount,TotalFormulaDiscount,TotalDiscount,TotalPrice,TotalCredit,NbrDaysSub,AvgDurationPerSub,AverageNbrMealPerDay,NbrCustomEventsProduct,NbrGrubProduct,label,Region_dum,StreetID_dum,features
923623,5,43628,1,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1520,304.0,70,14.0,1824.0,5,5,0,1.0,15.912104000000003,0.0,0.0,0.0,24186.4,-44.0,1820.0,364.0,0.835167356512313,0,5,0,"List(0, 8, List(0), List(1.0))","List(0, 1282, List(1256), List(1.0))","List(0, 1338, List(0, 1, 2, 11, 13, 24, 28, 29, 30, 31, 32, 33, 34, 36, 37, 41, 42, 43, 44, 45, 47, 48, 1312), List(5.0, 43628.0, 1.0, 1.0, 1.0, 1.0, 1520.0, 304.0, 70.0, 14.0, 1824.0, 5.0, 5.0, 1.0, 15.912104000000003, 24186.4, -44.0, 1820.0, 364.0, 0.835167356512313, 5.0, 1.0, 1.0))"
644766,1,102093,7,0.004844290657439446,0,0,0,0,0,7,0,0,0,6,0,1,0,0,0,0,0,0,0,3,1,1,0,2,1849,264.14285714285717,105,15.0,1779.0,7,5,2,0.7142857142857143,16.2117,0.0,0.0,0.0,22029.8,-370.2,1743.0,249.0,65.71642205702143,0,7,0,"List(0, 8, List(1), List(1.0))","List(0, 1282, List(1056), List(1.0))","List(0, 1338, List(0, 1, 2, 3, 9, 13, 15, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 41, 42, 43, 44, 45, 47, 49, 1112), List(1.0, 102093.0, 7.0, 0.004844290657439446, 7.0, 6.0, 1.0, 3.0, 1.0, 1.0, 2.0, 1849.0, 264.14285714285717, 105.0, 15.0, 1779.0, 7.0, 5.0, 2.0, 0.7142857142857143, 16.2117, 22029.8, -370.2, 1743.0, 249.0, 65.71642205702143, 7.0, 1.0, 1.0))"
80187,5,38209,1,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1545,309.0,85,17.0,1856.0,5,5,0,1.0,16.073668,0.0,376.6,376.6,24780.0,0.0,1852.0,370.4,0.8342216951395729,0,5,0,"List(0, 8, List(0), List(1.0))","List(0, 1282, List(3), List(1.0))","List(0, 1338, List(0, 1, 2, 11, 13, 23, 28, 29, 30, 31, 32, 33, 34, 36, 37, 39, 40, 41, 43, 44, 45, 47, 48, 59), List(5.0, 38209.0, 1.0, 1.0, 1.0, 1.0, 1545.0, 309.0, 85.0, 17.0, 1856.0, 5.0, 5.0, 1.0, 16.073668, 376.6, 376.6, 24780.0, 1852.0, 370.4, 0.8342216951395729, 5.0, 1.0, 1.0))"
285977,1,18415,7,0.0045721750489875895,0,0,0,0,0,7,0,0,0,4,1,1,0,0,0,0,0,1,0,1,0,1,0,5,1519,303.8,70,14.0,1860.0,5,5,0,1.0,15.705696,0.0,0.0,0.0,23858.199999999997,-721.8000000000001,1856.0,371.2,0.818706760989113,0,5,0,"List(0, 8, List(1), List(1.0))","List(0, 1282, List(1179), List(1.0))","List(0, 1338, List(0, 1, 2, 3, 9, 13, 14, 15, 21, 23, 25, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 41, 42, 43, 44, 45, 47, 49, 1235), List(1.0, 18415.0, 7.0, 0.0045721750489875895, 7.0, 4.0, 1.0, 1.0, 1.0, 1.0, 1.0, 5.0, 1519.0, 303.8, 70.0, 14.0, 1860.0, 5.0, 5.0, 1.0, 15.705696, 23858.199999999997, -721.8000000000001, 1856.0, 371.2, 0.818706760989113, 5.0, 1.0, 1.0))"
925863,5,43843,1,0.0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1519,303.8,70,14.0,1826.0,5,5,0,1.0,15.920856,0.0,0.0,0.0,24186.4,-44.0,1822.0,364.4,0.8337001354809574,0,5,0,"List(0, 8, List(0), List(1.0))","List(0, 1282, List(203), List(1.0))","List(0, 1338, List(0, 1, 2, 11, 13, 23, 28, 29, 30, 31, 32, 33, 34, 36, 37, 41, 42, 43, 44, 45, 47, 48, 259), List(5.0, 43843.0, 1.0, 1.0, 1.0, 1.0, 1519.0, 303.8, 70.0, 14.0, 1826.0, 5.0, 5.0, 1.0, 15.920856, 24186.4, -44.0, 1822.0, 364.4, 0.8337001354809574, 5.0, 1.0, 1.0))"
757625,5,41196,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1519,303.8,70,14.0,1825.0,5,5,0,1.0,16.283628,0.0,0.0,0.0,24736.0

In [35]:
preds = cvModel.transform(test)\
  .select("prediction", "label")
preds.show(10)

#Get model performance on test set
from pyspark.mllib.evaluation import BinaryClassificationMetrics

out = preds.rdd.map(lambda x: (float(x[0]), float(x[1])))
metrics = BinaryClassificationMetrics(out)

print(metrics.areaUnderPR) #area under precision/recall curve
print(metrics.areaUnderROC)#area under Receiver Operating Characteristic curve

+----------+-----+
prediction|label|
+----------+-----+
 1.0| 1|
 1.0| 0|
 1.0| 0|
 1.0| 1|
 1.0| 0|
 1.0| 1|
 1.0| 0|
 1.0| 1|
 1.0| 1|
 0.0| 0|
+----------+-----+
only showing top 10 rows

0.75
0.5686274509803921

In [36]:
#Get more metrics
from pyspark.mllib.evaluation import MulticlassMetrics

#Cast a DF of predictions to an RDD to access RDD methods of MulticlassMetrics
preds_labels = cvModel.transform(test)\
  .select("prediction", "label")\
  .rdd.map(lambda x: (float(x[0]), float(x[1])))

metrics = MulticlassMetrics(preds_labels)

print("accuracy = %s" % metrics.accuracy)

accuracy = 0.7595628415300546

In [37]:
#Get more metrics
from pyspark.mllib.evaluation import MulticlassMetrics

labels = preds.rdd.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))

Class 0 precision = 1.0
Class 0 recall = 0.13725490196078433
Class 1 precision = 0.75
Class 1 recall = 1.0

In [38]:
#Select the best RF model
rfcBestModel = rfcModel.bestModel.stages[-1] #-1 means "get last stage in the pipeline"

#Get tuned number of trees
rfcBestModel.getNumTrees

Out[55]: 500

In [39]:
#Prettify feature importances
import pandas as pd
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))
  
ExtractFeatureImp(rfcBestModel.featureImportances, train, "features").head(20)

,idx,name,score
41,41,TotalPrice,0.117071
32,32,HasBeenClientForXDays,0.085446
43,43,NbrDaysSub,0.082592
31,31,MeanMeal_EXCEPPerSub,0.074673
34,34,SubPaid,0.060538
37,37,AvgPricePerMeal,0.059804
33,33,NbrSub,0.052510
28,28,TotalMeal_REG,0.050055
30,30,TotalMeal_EXCEP,0.047162
47,47,NbrGrubProduct,0.038351
